In [2]:
import json
from neo4j import GraphDatabase
from google import adk
from google.adk.agents import Agent
from google.adk.tools import FunctionTool

# Load config
with open("healthcare_config.json", "r") as f:
    config = json.load(f)

class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def run_query(self, query, parameters=None):
        with self.driver.session() as session:
            result = session.run(query, parameters or {})
            return [record.data() for record in result]

conn = Neo4jConnection(config["neo4j_uri"], config["neo4j_user"], config["neo4j_password"])
print("Connected to Neo4j!")

Connected to Neo4j!


## 1. Bed Management Agent Tools

In [3]:
def get_available_beds(department: str = None) -> str:
    """Get count of available beds, optionally filtered by department."""
    if department:
        query = """
        MATCH (d:Department {name: $dept})-[:HAS_BED]->(b:Bed {status: 'available'})
        RETURN d.name as department, count(b) as available_beds
        """
        result = conn.run_query(query, {"dept": department})
    else:
        query = """
        MATCH (d:Department)-[:HAS_BED]->(b:Bed {status: 'available'})
        RETURN d.name as department, count(b) as available_beds
        ORDER BY d.name
        """
        result = conn.run_query(query)
    return json.dumps(result, indent=2)

def get_bed_occupancy_rate() -> str:
    """Get bed occupancy rate by department."""
    query = """
    MATCH (d:Department)-[:HAS_BED]->(b:Bed)
    WITH d.name as dept, count(b) as total,
         sum(CASE WHEN b.status = 'occupied' THEN 1 ELSE 0 END) as occupied
    RETURN dept, total, occupied, 
           round(toFloat(occupied)/total * 100, 1) as occupancy_rate
    ORDER BY occupancy_rate DESC
    """
    result = conn.run_query(query)
    return json.dumps(result, indent=2)

def get_icu_status() -> str:
    """Get detailed ICU bed status."""
    query = """
    MATCH (d:Department {name: 'ICU'})-[:HAS_BED]->(b:Bed)
    RETURN b.status as status, count(b) as count
    ORDER BY b.status
    """
    result = conn.run_query(query)
    return json.dumps(result, indent=2)

# Test
print("ICU Status:")
print(get_icu_status())

ICU Status:
[
  {
    "status": "available",
    "count": 6
  },
  {
    "status": "maintenance",
    "count": 5
  },
  {
    "status": "occupied",
    "count": 9
  }
]


## 2. Patient Research Agent Tools

In [4]:
def get_patients_by_status(status: str) -> str:
    """Get patients by status (stable, critical, recovering, ready_for_discharge)."""
    query = """
    MATCH (p:Patient {status: $status})-[:ADMITTED_TO]->(b:Bed)<-[:HAS_BED]-(d:Department)
    RETURN p.name as patient, p.condition as condition, p.age as age, 
           d.name as department, p.admitDate as admit_date
    ORDER BY p.admitDate
    """
    result = conn.run_query(query, {"status": status})
    return json.dumps(result, indent=2)

def get_discharge_candidates() -> str:
    """Get patients ready for discharge planning."""
    query = """
    MATCH (p:Patient)-[:ADMITTED_TO]->(b:Bed)<-[:HAS_BED]-(d:Department)
    WHERE p.status IN ['ready_for_discharge', 'recovering']
    RETURN p.name as patient, p.status as status, p.condition as condition,
           d.name as department, p.admitDate as admit_date
    ORDER BY p.status DESC, p.admitDate
    """
    result = conn.run_query(query)
    return json.dumps(result, indent=2)

def get_patients_by_condition(condition: str) -> str:
    """Get patients with a specific condition."""
    query = """
    MATCH (p:Patient)-[:ADMITTED_TO]->(b:Bed)<-[:HAS_BED]-(d:Department)
    WHERE toLower(p.condition) CONTAINS toLower($condition)
    RETURN p.name as patient, p.condition as condition, p.status as status,
           d.name as department
    """
    result = conn.run_query(query, {"condition": condition})
    return json.dumps(result, indent=2)

# Test
print("Discharge Candidates:")
print(get_discharge_candidates())

Discharge Candidates:
[
  {
    "patient": "Patient 15",
    "status": "recovering",
    "condition": "Appendicitis",
    "department": "Emergency",
    "admit_date": "2025-12-31T17:51:42.613446"
  },
  {
    "patient": "Patient 29",
    "status": "recovering",
    "condition": "Fracture",
    "department": "Emergency",
    "admit_date": "2025-12-31T17:51:42.746455"
  },
  {
    "patient": "Patient 31",
    "status": "recovering",
    "condition": "COVID-19",
    "department": "Cardiology",
    "admit_date": "2025-12-31T17:51:42.762472"
  },
  {
    "patient": "Patient 51",
    "status": "recovering",
    "condition": "Pneumonia",
    "department": "Pediatrics",
    "admit_date": "2026-01-01T17:51:42.901531"
  },
  {
    "patient": "Patient 52",
    "status": "recovering",
    "condition": "Fracture",
    "department": "Pediatrics",
    "admit_date": "2026-01-01T17:51:42.908253"
  },
  {
    "patient": "Patient 62",
    "status": "recovering",
    "condition": "Heart Failure",
    "dep

## 3. Inventory Agent Tools

In [5]:
def get_low_stock_items() -> str:
    """Get inventory items that are running low (below minimum stock)."""
    query = """
    MATCH (d:Department)-[:STOCKS]->(i:InventoryItem)
    WHERE i.quantity < i.minStock
    RETURN i.name as item, i.type as type, i.quantity as current_qty,
           i.minStock as min_required, d.name as department,
           i.minStock - i.quantity as shortage
    ORDER BY shortage DESC
    """
    result = conn.run_query(query)
    return json.dumps(result, indent=2)

def get_equipment_status() -> str:
    """Get status of all equipment items."""
    query = """
    MATCH (d:Department)-[:STOCKS]->(i:InventoryItem {type: 'equipment'})
    RETURN i.name as equipment, i.quantity as available, d.name as department,
           CASE WHEN i.quantity < i.minStock THEN 'LOW' ELSE 'OK' END as status
    ORDER BY d.name
    """
    result = conn.run_query(query)
    return json.dumps(result, indent=2)

def get_medication_inventory() -> str:
    """Get all medication inventory levels."""
    query = """
    MATCH (d:Department)-[:STOCKS]->(i:InventoryItem {type: 'medication'})
    RETURN i.name as medication, i.quantity as quantity, i.minStock as min_stock,
           d.name as department
    ORDER BY i.quantity
    """
    result = conn.run_query(query)
    return json.dumps(result, indent=2)

# Test
print("Low Stock Items:")
print(get_low_stock_items())

Low Stock Items:
[
  {
    "item": "Surgical Masks",
    "type": "supplies",
    "current_qty": 200,
    "min_required": 500,
    "department": "Surgery",
    "shortage": 300
  },
  {
    "item": "IV Bags",
    "type": "supplies",
    "current_qty": 50,
    "min_required": 100,
    "department": "ICU",
    "shortage": 50
  },
  {
    "item": "Blood Units (O-)",
    "type": "supplies",
    "current_qty": 12,
    "min_required": 25,
    "department": "Surgery",
    "shortage": 13
  },
  {
    "item": "Insulin",
    "type": "medication",
    "current_qty": 45,
    "min_required": 50,
    "department": "Cardiology",
    "shortage": 5
  }
]


## 4. Physician Agent Tools

In [6]:
def get_physicians_on_call() -> str:
    """Get all physicians currently on call."""
    query = """
    MATCH (p:Physician {onCall: true})-[:ASSIGNED_TO]->(d:Department)
    RETURN p.name as physician, p.specialty as specialty, d.name as department
    ORDER BY d.name
    """
    result = conn.run_query(query)
    return json.dumps(result, indent=2)

def get_physicians_by_department(department: str) -> str:
    """Get physicians assigned to a specific department."""
    query = """
    MATCH (p:Physician)-[:ASSIGNED_TO]->(d:Department {name: $dept})
    RETURN p.name as physician, p.specialty as specialty, p.onCall as on_call
    """
    result = conn.run_query(query, {"dept": department})
    return json.dumps(result, indent=2)

def get_all_physicians() -> str:
    """Get list of all physicians with their assignments."""
    query = """
    MATCH (p:Physician)-[:ASSIGNED_TO]->(d:Department)
    RETURN p.name as physician, p.specialty as specialty, 
           d.name as department, p.onCall as on_call
    ORDER BY d.name, p.name
    """
    result = conn.run_query(query)
    return json.dumps(result, indent=2)

# Test
print("Physicians On Call:")
print(get_physicians_on_call())

Physicians On Call:
[
  {
    "physician": "Dr. Robert Taylor",
    "specialty": "Cardiology",
    "department": "Cardiology"
  },
  {
    "physician": "Dr. James Wilson",
    "specialty": "Emergency Medicine",
    "department": "Emergency"
  },
  {
    "physician": "Dr. Sarah Chen",
    "specialty": "Critical Care",
    "department": "ICU"
  },
  {
    "physician": "Dr. Michael Brown",
    "specialty": "Pediatrics",
    "department": "Pediatrics"
  }
]


## 5. Graph Database Agent Tools

In [7]:
def execute_cypher_query(query: str) -> str:
    """Execute a raw Cypher query against the database. Use for custom queries."""
    try:
        result = conn.run_query(query)
        return json.dumps(result, indent=2)
    except Exception as e:
        return json.dumps({"error": str(e)})

def get_database_schema() -> str:
    """Get the database schema including node labels and relationships."""
    labels_query = "CALL db.labels() YIELD label RETURN collect(label) as labels"
    rels_query = "CALL db.relationshipTypes() YIELD relationshipType RETURN collect(relationshipType) as relationships"
    
    labels = conn.run_query(labels_query)
    rels = conn.run_query(rels_query)
    
    schema = {
        "node_labels": labels[0]["labels"] if labels else [],
        "relationship_types": rels[0]["relationships"] if rels else []
    }
    return json.dumps(schema, indent=2)

# Test
print("Database Schema:")
print(get_database_schema())

Database Schema:
{
  "node_labels": [
    "Hospital",
    "Department",
    "Bed",
    "Patient",
    "Physician",
    "InventoryItem"
  ],
  "relationship_types": [
    "HAS_DEPARTMENT",
    "HAS_BED",
    "ASSIGNED_TO",
    "ADMITTED_TO",
    "STOCKS"
  ]
}


## 6. Export Tools for Part 4

In [8]:
# Save all tool functions to a module file for Part 4
tools_code = '''
# Healthcare Agent Tools
import json
from neo4j import GraphDatabase

# Connection will be initialized in Part 4
conn = None

def init_connection(uri, user, password):
    global conn
    class Neo4jConn:
        def __init__(self, uri, user, password):
            self.driver = GraphDatabase.driver(uri, auth=(user, password))
        def run_query(self, query, parameters=None):
            with self.driver.session() as session:
                result = session.run(query, parameters or {})
                return [record.data() for record in result]
    conn = Neo4jConn(uri, user, password)
    return conn

# Bed Management Tools
def get_available_beds(department: str = None) -> str:
    if department:
        query = "MATCH (d:Department {name: $dept})-[:HAS_BED]->(b:Bed {status: \'available\'}) RETURN d.name as department, count(b) as available_beds"
        result = conn.run_query(query, {"dept": department})
    else:
        query = "MATCH (d:Department)-[:HAS_BED]->(b:Bed {status: \'available\'}) RETURN d.name as department, count(b) as available_beds ORDER BY d.name"
        result = conn.run_query(query)
    return json.dumps(result, indent=2)

def get_icu_status() -> str:
    query = "MATCH (d:Department {name: \'ICU\'})-[:HAS_BED]->(b:Bed) RETURN b.status as status, count(b) as count ORDER BY b.status"
    return json.dumps(conn.run_query(query), indent=2)

def get_discharge_candidates() -> str:
    query = "MATCH (p:Patient)-[:ADMITTED_TO]->(b:Bed)<-[:HAS_BED]-(d:Department) WHERE p.status IN [\'ready_for_discharge\', \'recovering\'] RETURN p.name as patient, p.status as status, p.condition as condition, d.name as department ORDER BY p.status DESC"
    return json.dumps(conn.run_query(query), indent=2)

def get_low_stock_items() -> str:
    query = "MATCH (d:Department)-[:STOCKS]->(i:InventoryItem) WHERE i.quantity < i.minStock RETURN i.name as item, i.type as type, i.quantity as current_qty, i.minStock as min_required, d.name as department ORDER BY (i.minStock - i.quantity) DESC"
    return json.dumps(conn.run_query(query), indent=2)

def get_physicians_on_call() -> str:
    query = "MATCH (p:Physician {onCall: true})-[:ASSIGNED_TO]->(d:Department) RETURN p.name as physician, p.specialty as specialty, d.name as department"
    return json.dumps(conn.run_query(query), indent=2)

def execute_cypher_query(query: str) -> str:
    try:
        return json.dumps(conn.run_query(query), indent=2)
    except Exception as e:
        return json.dumps({"error": str(e)})
'''

with open("healthcare_tools.py", "w") as f:
    f.write(tools_code)

print("Tools exported to healthcare_tools.py")
print("Proceed to Part 4 for the orchestrator agent.")

Tools exported to healthcare_tools.py
Proceed to Part 4 for the orchestrator agent.
